# 🧹 03 - Enrichissement des données OpenFoodFacts
Ce notebook a pour objectif d'enrichir les données déjà transformées en CSV et nettoyées.


In [1]:
# 0. Stoppe toute session existante
try:
    spark.stop()
except:
    pass

# Recréation SparkSession en local, FS local et driver binding fixe
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
        .appName("03_Enrichment")
        .master("local[*]")
        .config("spark.hadoop.fs.defaultFS", "file:///")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "0.0.0.0")
        .getOrCreate()
)


In [2]:
# Chargement des données nettoyées depuis le CSV généré à l'étape précédente
input_path = "../data/step2_cleaned_csv"

df_cleaned = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("sep", ";")
        .csv(input_path)
)

print(f"✅ Données chargées : {df_cleaned.count():,} lignes, {len(df_cleaned.columns)} colonnes")


✅ Données chargées : 3,247,216 lignes, 161 colonnes


In [3]:
#Conversion de energy_100g en kcal
from pyspark.sql.functions import when, regexp_replace , col

df_enriched  = df_cleaned \
    .withColumn("energy_kcal_100g", when(col("energy_100g").rlike("^[0-9]+$"),
                                         col("energy_100g") / 4.184)
                .otherwise(
                    when(col("energy_100g").rlike("^[0-9]+\\.?[0-9]* kcal$"),
                         regexp_replace(col("energy_100g"), " kcal", "").cast("double"))
                )) \
    .withColumn("energy_100g", when(col("energy_100g").rlike("^[0-9]+$"),
                                    col("energy_100g").cast("double")))


In [4]:
#Catégories nutritionnelles personnalisées

# profil énergétique
df_enriched  = df_enriched .withColumn("profil_energetique",
    when(col("energy-kcal_100g") < 100, "🔵 Faible")
    .when(col("energy-kcal_100g") < 250, "🟡 Modérée")
    .otherwise("🔴 Élevée")
)

# score sucré
df_enriched  = df_enriched .withColumn("profil_sucre",
    when(col("sugars_100g") < 5, "Peu sucré")
    .when(col("sugars_100g") < 15, "Modéré")
    .otherwise("Très sucré")
)


In [5]:
#Tags nutritionnels (booléens)
df_enriched  = df_enriched .withColumn("is_vegan", col("ingredients_tags").contains("vegan"))
df_enriched  = df_enriched .withColumn("is_vegetarian", col("ingredients_tags").contains("vegetarian"))
df_enriched  = df_enriched .withColumn("is_sans_sucre", col("sugars_100g") < 0.5)
df_enriched  = df_enriched .withColumn("is_protein_plus", col("proteins_100g") > 10)
df_enriched  = df_enriched .withColumn("is_light", col("fat_100g") < 3)
df_enriched = df_enriched.withColumn("is_ultra_transformed", col("nova_group") == 4)


In [6]:
# profil alimentaire
from pyspark.sql.functions import when, col, expr

# 1. Convertir nutriscore_grade (a-e) en score numérique (5-1)
df_enriched = df_enriched.withColumn("nutriscore_value",
    when(col("nutriscore_grade") == "a", 5)
    .when(col("nutriscore_grade") == "b", 4)
    .when(col("nutriscore_grade") == "c", 3)
    .when(col("nutriscore_grade") == "d", 2)
    .when(col("nutriscore_grade") == "e", 1)
    .otherwise(None)
)

# 2. Convertir nova_group (1-4) en score nutritionnel (4-1)
df_enriched = df_enriched.withColumn("nova_score",
    when(col("nova_group") == 1, 4)
    .when(col("nova_group") == 2, 3)
    .when(col("nova_group") == 3, 2)
    .when(col("nova_group") == 4, 1)
    .otherwise(None)
)

# 3. Score composite : combinaison pondérée (40% NutriScore, 60% NOVA)
df_enriched = df_enriched.withColumn("score_composite",
    expr("nutriscore_value * 0.4 + nova_score * 0.6")
)

# 4. Étiquette lisible pour le score global
df_enriched = df_enriched.withColumn("profil_global",
    when(col("score_composite").isNull(), None)
    .when(col("score_composite") >= 4.0, "🥦 Très sain")
    .when(col("score_composite") >= 3.0, "🟢 Assez sain")
    .when(col("score_composite") >= 2.0, "🟠 Moyennement sain")
    .otherwise("🔴 Peu sain")
)

# 🔍 Vérification
df_enriched \
    .filter(col("nutriscore_value").isNotNull() & col("nova_score").isNotNull()) \
    .select("nutriscore_grade", "nova_group", "nutriscore_value", "nova_score", "score_composite", "profil_global") \
    .show(10, truncate=False)
count_non_null = df_enriched.filter(col("profil_global").isNotNull()).count()
print(f"✅ Nombre de lignes avec un profil global non vide : {count_non_null:,}")

+----------------+----------+----------------+----------+---------------+-------------------+
|nutriscore_grade|nova_group|nutriscore_value|nova_score|score_composite|profil_global      |
+----------------+----------+----------------+----------+---------------+-------------------+
|a               |1         |5               |4         |4.4            |🥦 Très sain       |
|d               |4         |2               |1         |1.4            |🔴 Peu sain        |
|c               |4         |3               |1         |1.8            |🔴 Peu sain        |
|e               |4         |1               |1         |1.0            |🔴 Peu sain        |
|c               |4         |3               |1         |1.8            |🔴 Peu sain        |
|e               |4         |1               |1         |1.0            |🔴 Peu sain        |
|a               |3         |5               |2         |3.2            |🟢 Assez sain      |
|c               |4         |3               |1         |1.8       

In [7]:
# Impact environnemental 
from pyspark.sql.functions import col, when, expr

# Recrée la colonne empreinte_carbone_score si elle a été perdue
df_enriched = df_enriched.withColumn("empreinte_carbone_score",
    when(col("carbon-footprint_100g") <= 1, 5)
    .when(col("carbon-footprint_100g") <= 5, 4)
    .when(col("carbon-footprint_100g") <= 10, 3)
    .when(col("carbon-footprint_100g") <= 20, 2)
    .when(col("carbon-footprint_100g").isNotNull(), 1)
)

# Recrée l'eco_score
df_enriched = df_enriched.withColumn("eco_score",
    when(col("environmental_score_score").isNotNull(),
         6 - (col("environmental_score_score") / 10))
)

# Score environnemental flexible : accepte même s'il manque une des deux métriques
df_enriched = df_enriched.withColumn("score_env_composite_flexible",
    when(col("empreinte_carbone_score").isNotNull() & col("eco_score").isNotNull(),
         expr("0.5 * empreinte_carbone_score + 0.5 * eco_score"))
    .when(col("empreinte_carbone_score").isNotNull(), col("empreinte_carbone_score") * 1.0)
    .when(col("eco_score").isNotNull(), col("eco_score") * 1.0)
)

# Profil écologique lisible
df_enriched = df_enriched.withColumn("profil_ecolo_flexible",
    when(col("score_env_composite_flexible") >= 4.0, "♻️ Très faible impact")
    .when(col("score_env_composite_flexible") >= 3.0, "🟢 Faible impact")
    .when(col("score_env_composite_flexible") >= 2.0, "🟠 Impact modéré")
    .when(col("score_env_composite_flexible").isNotNull(), "🔴 Fort impact")
)

# 🔍 Vérification + Comptage
df_enriched \
    .filter(col("score_env_composite_flexible").isNotNull()) \
    .select("carbon-footprint_100g", "environmental_score_score", "empreinte_carbone_score", "eco_score", "score_env_composite_flexible", "profil_ecolo_flexible") \
    .show(10, truncate=False)
count_flexible = df_enriched.filter(col("score_env_composite_flexible").isNotNull()).count()
print(f"✅ Nombre de lignes avec un profil écolo (flexible) non vide : {count_flexible:,}")

✅ Nombre de lignes avec un profil écolo (flexible) non vide : 651,536


In [8]:
# 💾 Sauvegarde des données enrichies (CSV)
import os, shutil

# 1. Préparer le dossier de sortie
folder = os.path.abspath(os.path.join(os.getcwd(), "../data"))
os.makedirs(folder, exist_ok=True)

# 2. Chemin vers le nouveau dossier CSV
csv_target = os.path.join(folder, "step3_enriched_csv")

# 3. Supprimer l'ancienne sortie si elle existe
if os.path.exists(csv_target):
    shutil.rmtree(csv_target)
    print("🗑️ Ancienne sortie supprimée :", csv_target)

# 4. Écriture au format CSV avec en-têtes
df_enriched \
    .coalesce(1) \
    .write \
    .option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .csv(csv_target)

print("✅ Données enrichies écrites en CSV dans :", csv_target)


🗑️ Ancienne sortie supprimée : /home/jovyan/work/data/step3_enriched_csv
✅ Données enrichies écrites en CSV dans : /home/jovyan/work/data/step3_enriched_csv
